In [231]:
import pandas as pd
import numpy as np

In [232]:
train = pd.read_csv('option_train.csv')
test = pd.read_csv('option_test_wolabel.csv')

In [234]:
train = train.replace({'BS': {'Under':0, 'Over':1}})

In [238]:
new = train.copy()
new.drop(['BS','Value'], axis=1, inplace=True)

In [239]:
full_model = pd.concat([new, test])

In [240]:
v = train.copy()

# Creating Variables

In [241]:
col = list(new.columns)
col_r = col[::-1]

In [242]:
col

['S', 'K', 'tau', 'r']

In [243]:
for i in new.columns:
    for j in col_r:
        if i==j:
            col_r.remove(i)
        else:
            new[i+'_times_'+j] = new[i]*new[j]
            new[i+'_by_'+j] = new[i]/new[j]
            test[i+'_times_'+j] = test[i]*test[j]
            test[i+'_by_'+j] = test[i]/test[j]

In [244]:
new['Ke^-rt'] = new['K']*np.e**(-new['r']*new['tau'])
new['Ke^-Kt'] = new['K']*np.e**(-new['K']*new['tau'])
test['Ke^-rt'] = test['K']*np.e**(-test['r']*test['tau'])
test['Ke^-Kt'] = test['K']*np.e**(-test['K']*test['tau'])
new['S_minus_K'] = new['S'] - new['K']
test['S_minus_K'] = test['S'] - test['K']

In [245]:
new = new.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

In [246]:
new.head()

,S,K,tau,r,S_times_r,S_by_r,S_times_tau,S_by_tau,S_times_K,S_by_K,K_times_r,K_by_r,K_times_tau,K_by_tau,tau_times_r,tau_by_r,Ke^-rt,Ke^-Kt,S_minus_K
0,431.623898,420,0.341270,0.03013,13.004828,14325.386601,147.300219,1264.758401,181282.037286,1.027676,12.6546,13939.595088,143.333333,1230.697675,0.010282,11.326580,415.703494,2.367956e-60,11.623898
1,427.015526,465,0.166667,0.03126,13.348505,13660.125589,71.169254,2562.093150,198562.219543,0.918313,14.5359,14875.239923,77.500000,2789.999994,0.005210,5.331627,462.583650,1.022423e-31,-37.984474
2,427.762336,415,0.265873,0.03116,13.329074,13727.931207,113.730463,1608.897145,177521.369606,1.030753,12.9314,13318.356868,110.337302,1560.895522,0.008285,8.532510,411.576092,5.002267e-46,12.762336
3,451.711658,460,0.063492,0.02972,13.424870,15198.911770,28.680105,7114.458665,207787.362588,0.981982,13.6712,15477.792732,29.206349,7245.000056,0.001887,2.136341,459.132806,9.519226e-11,-8.288342
4,446.718974,410,0.166667,0.02962,13.231816,15081.666928,74.453163,2680.313841,183154.779504,1.089558,12.1442,13841.998650,68.333333,2459.999995,0.004937,5.626829,407.980954,8.629670e-28,36.718974


In [207]:
# end = len(v.columns)
# for i in range(end):
#     for j in range(i+1,end):
#         v[i,j] = v.iloc[:,i]*v.iloc[:,j]

# Model

## Feature Selection

### RFE

In [166]:
train_final = new.copy()

In [167]:
X = train_final
y_regression = train['Value']
y = train['BS']

In [168]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
model = LinearRegression()
rfe = RFE(model, n_features_to_select=8)
rfe.fit(X,y_regression)

RFE(estimator=LinearRegression(), n_features_to_select=8)

In [169]:
var_selected = pd.DataFrame(sorted(zip(map(lambda x: round(x), rfe.ranking_), X.columns)),
                           columns = ['ranking','variable'])
print(var_selected)

    ranking     variable
0         1    K_times_r
1         1       S_by_K
2         1    S_minus_K
3         1    S_times_r
4         1            r
5         1          tau
6         1     tau_by_r
7         1  tau_times_r
8         2       Ke^-rt
9         3            S
10        4            K
11        5  K_times_tau
12        6       K_by_r
13        7       S_by_r
14        8  S_times_tau
15        9       Ke^-Kt
16       10    S_times_K
17       11     K_by_tau
18       12     S_by_tau


In [170]:
rfe_selected = train_final[var_selected.head(6)['variable'].values]

In [171]:
rfe_selected

,K_times_r,S_by_K,S_minus_K,S_times_r,r,tau
0,12.65460,1.027676,11.623898,13.004828,0.03013,0.341270
1,14.53590,0.918313,-37.984474,13.348505,0.03126,0.166667
2,12.93140,1.030753,12.762336,13.329074,0.03116,0.265873
3,13.67120,0.981982,-8.288342,13.424870,0.02972,0.063492
4,12.14420,1.089558,36.718974,13.231816,0.02962,0.166667
...,...,...,...,...,...,...
1675,14.41150,0.935573,-30.602837,13.483010,0.03034,0.273810
1676,12.18520,1.069400,28.453825,13.030848,0.02972,0.301587
1677,12.43320,1.097544,38.042219,13.645986,0.03188,0.170635
1678,14.21760,0.914753,-40.918797,13.005585,0.02962,0.293651


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(rfe_selected,y_regression,test_size=0.20,random_state = 1)

### Forward Selection

In [25]:
#works terribly?

In [95]:
#importing the necessary libraries
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression
# Sequential Forward Selection(sfs)
sfs = SFS(LogisticRegression(max_iter=1700),
          k_features=18,
          forward=True,
          floating=True,
          scoring = 'accuracy',
          cv = 10)

In [96]:
sfs.fit(X, y)
sfs_fwd = list(sfs.k_feature_names_)


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

In [94]:
sfs_fwd

['S',
 'r',
 'S_times_r',
 'S_times_tau',
 'S_by_tau',
 'K_times_r',
 'K_times_tau',
 'tau_by_r',
 'Ke^-Kt',
 'S_minus_K']

In [29]:
fwd_selected = train_final[sfs_fwd]
fwd_selected

,S,r,S_times_r,S_times_tau,S_by_tau,K_times_r,tau_by_r,S_minus_K
0,431.623898,0.03013,13.004828,147.300219,1264.758401,12.65460,11.326580,11.623898
1,427.015526,0.03126,13.348505,71.169254,2562.093150,14.53590,5.331627,-37.984474
2,427.762336,0.03116,13.329074,113.730463,1608.897145,12.93140,8.532510,12.762336
3,451.711658,0.02972,13.424870,28.680105,7114.458665,13.67120,2.136341,-8.288342
4,446.718974,0.02962,13.231816,74.453163,2680.313841,12.14420,5.626829,36.718974
...,...,...,...,...,...,...,...,...
1675,444.397163,0.03034,13.483010,121.680176,1623.015726,14.41150,9.024704,-30.602837
1676,438.453825,0.02972,13.030848,132.232106,1453.820576,12.18520,10.147621,28.453825
1677,428.042219,0.03188,13.645986,73.038950,2508.526487,12.43320,5.352413,38.042219
1678,439.081203,0.02962,13.005585,128.936544,1495.249499,14.21760,9.913936,-40.918797


In [30]:
train_final.head()

,S,K,tau,r,S_times_r,S_by_r,S_times_tau,S_by_tau,S_times_K,S_by_K,K_times_r,K_by_r,K_times_tau,K_by_tau,tau_times_r,tau_by_r,Ke^-rt,Ke^-Kt,S_minus_K
0,431.623898,420,0.341270,0.03013,13.004828,14325.386601,147.300219,1264.758401,181282.037286,1.027676,12.6546,13939.595088,143.333333,1230.697675,0.010282,11.326580,415.703494,2.367956e-60,11.623898
1,427.015526,465,0.166667,0.03126,13.348505,13660.125589,71.169254,2562.093150,198562.219543,0.918313,14.5359,14875.239923,77.500000,2789.999994,0.005210,5.331627,462.583650,1.022423e-31,-37.984474
2,427.762336,415,0.265873,0.03116,13.329074,13727.931207,113.730463,1608.897145,177521.369606,1.030753,12.9314,13318.356868,110.337302,1560.895522,0.008285,8.532510,411.576092,5.002267e-46,12.762336
3,451.711658,460,0.063492,0.02972,13.424870,15198.911770,28.680105,7114.458665,207787.362588,0.981982,13.6712,15477.792732,29.206349,7245.000056,0.001887,2.136341,459.132806,9.519226e-11,-8.288342
4,446.718974,410,0.166667,0.02962,13.231816,15081.666928,74.453163,2680.313841,183154.779504,1.089558,12.1442,13841.998650,68.333333,2459.999995,0.004937,5.626829,407.980954,8.629670e-28,36.718974


#### My Selection

In [39]:
nush= train_final[['r','S_times_tau', 'K_times_r', 'tau_times_r', 'S_minus_K','Ke^-Kt']]

In [ ]:
nush2 = train_final[['S', 'K', 'tau', 'r', 'S_times_r', 'S_times_tau', 'S_times_K',
       'K_times_r', 'K_times_tau', 'tau_times_r', 'S_minus_K']]

In [33]:
nush.head()

,S,K,r,S_times_tau,K_times_r,tau_times_r,S_minus_K,Ke^-Kt
0,431.623898,420,0.03013,147.300219,12.6546,0.010282,11.623898,2.367956e-60
1,427.015526,465,0.03126,71.169254,14.5359,0.005210,-37.984474,1.022423e-31
2,427.762336,415,0.03116,113.730463,12.9314,0.008285,12.762336,5.002267e-46
3,451.711658,460,0.02972,28.680105,13.6712,0.001887,-8.288342,9.519226e-11
4,446.718974,410,0.02962,74.453163,12.1442,0.004937,36.718974,8.629670e-28


## Cross Validation

In [34]:
from sklearn.model_selection import KFold 

kfolds_r = KFold(n_splits = 10, random_state = 1, shuffle = True)

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

kfolds_regresssion = KFold(n_splits = 10, random_state = 1, shuffle = True)
regresssion_model = LinearRegression()
r2_model_1_cv = cross_val_score(regresssion_model,X,y_regression, cv=kfolds_regresssion)
r2_model_2_cv = cross_val_score(regresssion_model,rfe_selected,y_regression, cv=kfolds_regresssion)
#r2_model_3_cv = cross_val_score(regresssion_model,fwd_selected,y_regression, cv=kfolds_regresssion)
r2_model_4_cv = cross_val_score(regresssion_model,nush,y_regression, cv=kfolds_regresssion)

print("Linear Regression: \n")
print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")
print("r squared of 10-folds:",r2_model_2_cv,"(mean r squared:",np.mean(r2_model_2_cv),")")
#print("r squared of 10-folds:",r2_model_3_cv,"(mean r squared:",np.mean(r2_model_3_cv),")")
print("r squared of 10-folds:",r2_model_4_cv,"(mean r squared:",np.mean(r2_model_4_cv),")")

Linear Regression: 

r squared of 10-folds: [0.99446749 0.99429339 0.99383955 0.99143158 0.99456862 0.99123637
 0.99304574 0.99130611 0.99241621 0.99337315] (mean r squared: 0.9929978200970432 )
r squared of 10-folds: [0.989246   0.98704128 0.98582434 0.9824359  0.98848399 0.98514196
 0.98591701 0.98258043 0.98420896 0.98654811] (mean r squared: 0.9857427986459124 )
r squared of 10-folds: [0.91367832 0.91277457 0.90477328 0.89694054 0.91811225 0.93012067
 0.91646694 0.91738917 0.89869166 0.89936832] (mean r squared: 0.9108315719028915 )


In [36]:
X.columns

Index(['S', 'K', 'tau', 'r', 'S_times_r', 'S_by_r', 'S_times_tau', 'S_by_tau',
       'S_times_K', 'S_by_K', 'K_times_r', 'K_by_r', 'K_times_tau', 'K_by_tau',
       'tau_times_r', 'tau_by_r', 'Ke^-rt', 'Ke^-Kt', 'S_minus_K'],
      dtype='object')

In [108]:
#BEST

from sklearn.ensemble import RandomForestRegressor

kfolds_regresssion = KFold(n_splits = 10, random_state = 1, shuffle = True)
regresssion_model = RandomForestRegressor(n_estimators=450, random_state=1)
r2_model_1_cv = cross_val_score(regresssion_model,X,y_regression, cv=kfolds_regresssion)
r2_model_2_cv = cross_val_score(regresssion_model,rfe_selected,y_regression, cv=kfolds_regresssion)
#r2_model_3_cv = cross_val_score(regresssion_model,fwd_selected,y_regression, cv=kfolds_regresssion)
r2_model_4_cv = cross_val_score(regresssion_model,nush,y_regression, cv=kfolds_regresssion)

print("Linear Regression: \n")
print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")
print("r squared of 10-folds:",r2_model_2_cv,"(mean r squared:",np.mean(r2_model_2_cv),")") #use this!
#print("r squared of 10-folds:",r2_model_3_cv,"(mean r squared:",np.mean(r2_model_3_cv),")")
print("r squared of 10-folds:",r2_model_4_cv,"(mean r squared:",np.mean(r2_model_4_cv),")")

Linear Regression: 

r squared of 10-folds: [0.99880357 0.99863458 0.99846737 0.99827792 0.99800598 0.99874631
 0.99860729 0.99856049 0.99862265 0.99837572] (mean r squared: 0.9985101865388424 )
r squared of 10-folds: [0.99874549 0.99857111 0.99845697 0.99823982 0.99831692 0.99887917
 0.99869668 0.99863973 0.99869381 0.99837538] (mean r squared: 0.9985615060094043 )
r squared of 10-folds: [0.99884765 0.99864423 0.99863032 0.9984904  0.9985891  0.99901584
 0.99878527 0.99856014 0.9987749  0.99866926] (mean r squared: 0.9987007126719594 )


In [36]:
#iteration to find best
from sklearn.ensemble import RandomForestRegressor
kfolds_regresssion = KFold(n_splits = 10, random_state = 1, shuffle = True)

for i in range(100,600, 50):

    regresssion_model = RandomForestRegressor(n_estimators=i, random_state=1)


#r2_model_1_cv = cross_val_score(regresssion_model,X,y_regression, cv=kfolds_regresssion)
#r2_model_2_cv = cross_val_score(regresssion_model,rfe_selected,y_regression, cv=kfolds_regresssion)
#r2_model_3_cv = cross_val_score(regresssion_model,fwd_selected,y_regression, cv=kfolds_regresssion)
    r2_model_4_cv = cross_val_score(regresssion_model,nush,y_regression, cv=kfolds_regresssion)

    print("Linear Regression: \n")
#     print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")
#     print("r squared of 10-folds:",r2_model_2_cv,"(mean r squared:",np.mean(r2_model_2_cv),")")
#     print("r squared of 10-folds:",r2_model_3_cv,"(mean r squared:",np.mean(r2_model_3_cv),")")
    print("r squared of 10-folds:",i,"(mean r squared:",np.mean(r2_model_4_cv),")")

Linear Regression: 

r squared of 10-folds: 100 (mean r squared: 0.9985344460391399 )
Linear Regression: 

r squared of 10-folds: 150 (mean r squared: 0.9985513030864424 )
Linear Regression: 

r squared of 10-folds: 200 (mean r squared: 0.9985502893204078 )
Linear Regression: 

r squared of 10-folds: 250 (mean r squared: 0.9985600927973856 )
Linear Regression: 

r squared of 10-folds: 300 (mean r squared: 0.9985527348657339 )
Linear Regression: 

r squared of 10-folds: 350 (mean r squared: 0.9985540429686492 )
Linear Regression: 

r squared of 10-folds: 400 (mean r squared: 0.9985561043439024 )
Linear Regression: 

r squared of 10-folds: 450 (mean r squared: 0.9985572787260123 )
Linear Regression: 

r squared of 10-folds: 500 (mean r squared: 0.9985512576967427 )
Linear Regression: 

r squared of 10-folds: 550 (mean r squared: 0.9985524941064312 )


In [40]:
from sklearn.ensemble import GradientBoostingRegressor

kfolds_regresssion = KFold(n_splits = 10, random_state = 1, shuffle = True)
regresssion_model = GradientBoostingRegressor()
r2_model_1_cv = cross_val_score(regresssion_model,X,y_regression, cv=kfolds_regresssion)
r2_model_2_cv = cross_val_score(regresssion_model,rfe_selected,y_regression, cv=kfolds_regresssion)
#r2_model_3_cv = cross_val_score(regresssion_model,fwd_selected,y_regression, cv=kfolds_regresssion)
r2_model_4_cv = cross_val_score(regresssion_model,nush,y_regression, cv=kfolds_regresssion)

print("Linear Regression: \n")
print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")
print("r squared of 10-folds:",r2_model_2_cv,"(mean r squared:",np.mean(r2_model_2_cv),")")
#print("r squared of 10-folds:",r2_model_3_cv,"(mean r squared:",np.mean(r2_model_3_cv),")")
print("r squared of 10-folds:",r2_model_4_cv,"(mean r squared:",np.mean(r2_model_4_cv),")")

Linear Regression: 

r squared of 10-folds: [0.99854814 0.99838929 0.99859098 0.99838915 0.99855652 0.99828308
 0.99848237 0.99859385 0.99863336 0.99846397] (mean r squared: 0.9984930702690141 )
r squared of 10-folds: [0.99838797 0.99866534 0.99855444 0.99829983 0.99830522 0.99842601
 0.99843011 0.99846757 0.99851847 0.99852038] (mean r squared: 0.9984575342294697 )
r squared of 10-folds: [0.99879015 0.99869045 0.99866785 0.99817326 0.99871774 0.99839309
 0.99848539 0.99853167 0.99844878 0.99862663] (mean r squared: 0.9985525023282376 )


In [126]:
#iteration
from sklearn.ensemble import GradientBoostingRegressor


kfolds_regresssion = KFold(n_splits = 10, random_state = 1, shuffle = True)

for i in range(1000,6000, 500):
    regresssion_model = GradientBoostingRegressor(n_estimators=i, learning_rate=1,max_depth=1, random_state=1)
#     r2_model_1_cv = cross_val_score(regresssion_model,X,y_regression, cv=kfolds_regresssion)
#     r2_model_2_cv = cross_val_score(regresssion_model,rfe_selected,y_regression, cv=kfolds_regresssion)
#     r2_model_3_cv = cross_val_score(regresssion_model,fwd_selected,y_regression, cv=kfolds_regresssion)
    r2_model_4_cv = cross_val_score(regresssion_model,nush,y_regression, cv=kfolds_regresssion)

#     print("Linear Regression: \n")
#     print("r squared of 10-folds:",r2_model_1_cv,"(mean r squared:",np.mean(r2_model_1_cv),")")
#     print("r squared of 10-folds:",r2_model_2_cv,"(mean r squared:",np.mean(r2_model_2_cv),")")
#     print("r squared of 10-folds:",r2_model_3_cv,"(mean r squared:",np.mean(r2_model_3_cv),")")
    print("r squared of 10-folds:",i,"(mean r squared:",np.mean(r2_model_4_cv),")")

r squared of 10-folds: 1000 (mean r squared: 0.9965135385986373 )
r squared of 10-folds: 1500 (mean r squared: 0.996577256221253 )
r squared of 10-folds: 2000 (mean r squared: 0.9965690693570173 )
r squared of 10-folds: 2500 (mean r squared: 0.9965618203615444 )
r squared of 10-folds: 3000 (mean r squared: 0.9965403608787913 )
r squared of 10-folds: 3500 (mean r squared: 0.9965194451115551 )
r squared of 10-folds: 4000 (mean r squared: 0.9965040889860773 )
r squared of 10-folds: 4500 (mean r squared: 0.9964721031632067 )


KeyboardInterrupt: 

## Modeling

In [182]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [233]:
y_regression = train['Value']

In [183]:
X = fwd_selected

In [232]:
mms = MinMaxScaler()
X = mms.fit_transform(X)

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state = 1)

In [234]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X,y_regression,test_size=0.20,random_state = 1)

In [186]:
X_2 = rfe_selected.values

In [187]:
mms = MinMaxScaler()
X_2 = mms.fit_transform(X_2)

In [188]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X,y,test_size=0.20,random_state = 1)

# Logistic Regression 

In [189]:
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=1700).fit(X_train, y_train)

In [190]:
print(f'Classification Error: {1-mlr.score(X_test, y_test)}')

Classification Error: 0.0535714285714286


In [191]:
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=1700).fit(X_train_2, y_train_2)

In [192]:
print(f'Classification Error: {1-mlr.score(X_test_2, y_test_2)}')

Classification Error: 0.0535714285714286


# LDA

In [193]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [194]:
print(f'Classification Error: {1-lda.score(X_test, y_test)}')

Classification Error: 0.05654761904761907


In [195]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train_2, y_train)

LinearDiscriminantAnalysis()

In [196]:
print(f'Classification Error: {1-lda.score(X_test_2, y_test)}')

Classification Error: 0.05654761904761907


# KNN

### 5 neighbors

In [197]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [198]:
print(f'Classification Error: {1-knn.score(X_test, y_test)}')

Classification Error: 0.07738095238095233


### 10 neighbors

In [199]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [200]:
print(f'Classification Error: {1-knn.score(X_test, y_test)}')

Classification Error: 0.06547619047619047


# Boosted Trees

In [201]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0,
                                 max_depth=1, random_state=1).fit(X_train, y_train)

print(f'Classification Error: {1-clf.score(X_test, y_test)}')

Classification Error: 0.08333333333333337


In [202]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=1,
                                 max_depth=1, random_state=1).fit(X_train, y_train)

print(f'Classification Error: {1-clf.score(X_test, y_test)}')

Classification Error: 0.08630952380952384


# Random Forest

In [302]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100, random_state=1)
clf.fit(X_train,y_train)
print(f'Classification Error: {1-clf.score(X_test, y_test)}')

Classification Error: 0.0535714285714286


In [205]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=200, random_state=1)
clf.fit(X_train,y_train)
print(f'Classification Error: {1-clf.score(X_test, y_test)}')

Classification Error: 0.05059523809523814


# Random Forest Regression

In [269]:
from sklearn.ensemble import RandomForestRegressor
clf=RandomForestRegressor(n_estimators=100)
clf.fit(X_train_r,y_train_r)
print(f'Accuracy: {clf.score(X_test_r, y_test_r)}')

Accuracy: 0.9934577699489253


In [ ]:
kf = KFold(n_splits = 10, shuffle = True)

rf_table = pd.DataFrame(columns=['n_estimators', 'max_depth', 'X_set', 'Error'])

x_dict = {
    'X': X,
    'X_copy': X_copy,
    'X_origin': X_origin
}

row_index = 0
for i in range(50, 300, 50):
    for ii in [10, 15, 20, 30]:
        for x in ['X', 'X_copy', 'X_origin']:
            rf_table.loc[row_index, 'n_estimators'] = i
            rf_table.loc[row_index, 'max_depth'] = ii
            rf_table.loc[row_index, 'X_set'] = x

            rf = RandomForestClassifier(n_estimators=i,max_depth=ii)
            e = round(1-cross_val_score(rf, x_dict[x], y, scoring='accuracy', cv=kf).mean(), 4)
            rf_table.loc[row_index, 'Error'] = e
            print(f'Row {row_index} calculation finished.')
            row_index += 1
            

rf_table.sort_values("Error", ascending=True)

# Final Predictions

In [267]:
test=test[var_selected.head(6)['variable'].values]
y_oot = pd.read_csv('group_0_prediction.csv')

,Value,BS
0,1.0,0
1,2.0,0
2,1.0,0
3,1.0,0
4,3.0,0
...,...,...
1115,1.0,1
1116,1.0,1
1117,1.0,1
1118,1.0,1


In [265]:
from sklearn.ensemble import RandomForestRegressor

regresssion_model = RandomForestRegressor(n_estimators=450, random_state=1)
regresssion_model.fit(rfe_selected,y_regression)
regresssion_model.score(rfe_selected,y_regression)
predictions = regresssion_model.predict(test)

In [271]:
predictions=pd.DataFrame(predictions).rename(columns={0:'Value'})

In [273]:
predictions.to_csv('FinalPredictions.csv')